In [0]:
#colab 을 이용한 실행시
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchtext.data as data
import torchtext.datasets as datasets
import pickle
#print (torch.__version__)

In [0]:
class RNN_Text(nn.Module):
    
    def __init__(self, embed_num, class_num):
        super(RNN_Text, self).__init__()
        V = embed_num
        C = class_num
        H = 256
        
        self.embed = nn.Embedding(V, 100)
        self.rnn = nn.LSTM(100,H, bidirectional = True)
        self.out = nn.Linear(H*2, C)
        
    def forward(self, x):
        x = self.embed(x)  # (N, W, D)
        ##x = x.unsqueeze(1)  # (N, Ci, W, D)
        x,(_,__) = self.rnn( x, ( self.h, self.c ) )
        
        logit = self.out(x[-1])
        return logit
    def inithidden(self,b):
        self.h = Variable(torch.randn(2, b, 256))
        self.c = Variable(torch.randn(2, b, 256))
        

In [0]:
class mydataset(data.Dataset):
    @staticmethod
    def sort_key(ex):
        return len(ex.text)
    def __init__(self, text_field, label_field, path=None, examples=None, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        if examples is None:
            path = self.dirname if path is None else path
            examples = []
            for i,line in enumerate(open(path,'r',encoding='utf-8')):
                if i==0:
                    continue
                line = line.strip().split('\t')
                txt = line[1].split(' ')
                #txt= [ d.split('/')[0] for d in line[1].split(' ') ]
                examples += [ data.Example.fromlist( [txt, line[2]],fields ) ]
        super(mydataset, self).__init__(examples, fields, **kwargs)

In [0]:
text_field = data.Field(fix_length=20)
#text_field = data.Field()
label_field = data.Field(sequential=False, batch_first = True, unk_token = None)

train_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/Colab Notebooks/nsm/small_ratings_train_tok.txt')

test_data = mydataset(text_field,label_field,path='/content/gdrive/My Drive/Colab Notebooks/nsm/small_ratings_test_tok.txt')

text_field.build_vocab(train_data)
label_field.build_vocab(train_data)

train_iter, test_iter = data.Iterator.splits(
                            (train_data, test_data), 
                            batch_sizes=(100, 1), repeat=False)#, device = -1)
len(text_field.vocab)

21893

In [0]:
rnn = RNN_Text(len(text_field.vocab),2)
optimizer = torch.optim.Adam(rnn.parameters())
rnn.train()


RNN_Text(
  (embed): Embedding(21893, 100)
  (rnn): LSTM(100, 256, bidirectional=True)
  (out): Linear(in_features=512, out_features=2, bias=True)
)

In [0]:
%%time
# 실행 전에 구글 드라이브 Colab Notebooks 폴더에 model 폴더 생성
for epoch in range(15):
    z=0
    totalloss = 0
    for batch in train_iter:
        optimizer.zero_grad()
        
        txt = batch.text
        label = batch.label
        #print (txt.size())
        rnn.inithidden(txt.size(1))
        
        pred = rnn(txt)
        #print(pred.size(), label.size())
        #print(label)
        loss = F.cross_entropy(pred, label)
        totalloss += loss.data
        
        loss.backward()
        optimizer.step()
        #print(data,label)
        
    print(epoch,'epoch')  
    print(totalloss)
    
torch.save(rnn,'/content/gdrive/My Drive/Colab Notebooks/model/rnn_model.pt')

0 epoch
tensor(69.5946)
1 epoch
tensor(67.1414)
2 epoch
tensor(55.7175)
3 epoch
tensor(44.3771)
4 epoch
tensor(34.6700)
5 epoch
tensor(26.4957)
6 epoch
tensor(19.2615)
7 epoch
tensor(14.1647)
8 epoch
tensor(10.7159)
9 epoch
tensor(7.3472)
10 epoch
tensor(6.2484)
11 epoch
tensor(5.6280)
12 epoch
tensor(4.3078)
13 epoch
tensor(3.4514)
14 epoch
tensor(2.8498)
CPU times: user 5min 55s, sys: 4.49 s, total: 5min 59s
Wall time: 6min 1s


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN_Text. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
correct = 0
incorrect = 0
rnn.eval()
for batch in test_iter:
    txt = batch.text
    label = batch.label
    
    rnn.inithidden(txt.size(1))
    
    pred = rnn(txt)
    _,ans = torch.max(pred,dim=1)
    
    if ans.data[0] == label.data[0]:
        correct += 1    
    else:
        incorrect += 1
    
print ('correct : ', correct)
print ('incorrect : ', incorrect)
print ()


correct :  84
incorrect :  16

